In [1]:
!pip install scikit-surprise


In [2]:
!pip install pymongo
from pymongo import MongoClient
import numpy as np
import pandas as pd
import re
import ast
# !pip uninstall -y scikit-surprise numpy
# !pip install numpy==1.23.5 scikit-surprise

from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
import pickle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 kB 11.3 MB/s eta 0:00:00


In [ ]:

CONNECTION_STRING = "
# Connect to MongoDB
client = MongoClient(CONNECTION_STRING)

# Reference the correct databases
movie_db = client["movie_database"]
user_db = client["user_database"]


In [27]:
# def fetch_collection_data(db, collection_name):
#     """
#     Fetches data from a specified MongoDB collection.
#     """
#     collection = db[collection_name]
#     data = list(collection.find({}, {"_id": 0}))  # Exclude ObjectId
#     return pd.DataFrame(data)
def fetch_collection_data(db, collection_name):
    """
    Fetches data from a specified MongoDB collection while keeping '_id' as 'movie_id'.
    """
    collection = db[collection_name]
    data = list(collection.find({}))  # Do NOT exclude '_id'

    df = pd.DataFrame(data)

    # If '_id' exists, rename it to 'movie_id'
    # if '_id' in df.columns:
    #     df.rename(columns={'_id': 'movie_id'}, inplace=True)

    # Ensure movie_id is a string for consistency
    # df['movie_id'] = df['movie_id'].astype(str)

    return df



In [28]:
df = fetch_collection_data(movie_db, "movie_info")
df.head()

,_id,movie_id,adult,genres,original_language,release_date
0,67db28a0de0541f79e78e36b,finding+nemo+2003,False,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",en,2003-05-30
1,67db28bcde0541f79e78e36f,ong-bak+the+thai+warrior+2003,False,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",th,2003-01-21
2,67db28bcde0541f79e78e371,le+samoura+1967,False,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",fr,1967-10-25
3,67db28bcde0541f79e78e373,the+avengers+2012,False,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",en,2012-04-25
4,67db28bcde0541f79e78e375,the+lord+of+the+rings+the+return+of+the+king+2003,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,2003-12-01


In [29]:
df = fetch_collection_data(movie_db, "user_rate_data")
df.head()

,_id,time,user_id,movie_id,score
0,67db28c1facd51f1e29f0d4d,2025-03-19 20:27:40,71467,interstellar+2014,4.0
1,67db28cffacd51f1e29f0e06,2025-03-19 20:27:41,17425,milk+and+honey+2003,4.0
2,67db28d9facd51f1e29f0e7d,2025-03-19 20:27:41,29738,hamlet+1996,3.0
3,67db28e8facd51f1e29f0f5a,2025-03-19 20:27:42,136382,a+brief+history+of+time+1991,4.0
4,67db28e9facd51f1e29f0f6e,2025-03-19 20:27:42,123892,edmond+2005,2.0


In [30]:
df = fetch_collection_data(movie_db, "user_watch_data")
df.head()

,_id,time,user_id,movie_id,minute_mpg
0,67db28bcfacd51f1e29f0d11,2025-03-19 20:27:40,47697,ong-bak+the+thai+warrior+2003,7.mpg
1,67db28bcfacd51f1e29f0d12,2025-03-19 20:27:40,66358,le+samoura+1967,2.mpg
2,67db28bcfacd51f1e29f0d13,2025-03-19 20:27:40,62077,the+avengers+2012,32.mpg
3,67db28bcfacd51f1e29f0d14,2025-03-19 20:27:40,88449,the+lord+of+the+rings+the+return+of+the+king+2003,197.mpg
4,67db28bcfacd51f1e29f0d15,2025-03-19 20:27:40,152615,13+beloved+2006,56.mpg


In [31]:
df = fetch_collection_data(user_db, "user_info")
df.head()

,_id,user_id,age,gender,occupation
0,67db28a0de0541f79e78e36a,25436,27,M,sales/marketing
1,67db28bcde0541f79e78e36e,47697,32,M,college/grad student
2,67db28bcde0541f79e78e370,66358,30,F,sales/marketing
3,67db28bcde0541f79e78e372,62077,25,M,college/grad student
4,67db28bcde0541f79e78e374,88449,30,M,college/grad student


In [6]:
def clean_user_data():
    """
    Loads and cleans user data from MongoDB (user_database.user_info).
    """
    df = fetch_collection_data(user_db, "user_info")
    return df[['age', 'occupation', 'gender']]



In [7]:
def clean_ratings_data():
    """
    Loads and cleans ratings data from MongoDB (movie_database.user_rate_data).
    """
    df = fetch_collection_data(movie_db, "user_rate_data")

    # Fix movie_id formatting (replace '+' with space)
    df['movie_id'] = df['movie_id'].astype(str)

    return df[['user_id', 'movie_id', 'score']]



In [45]:
# def clean_movie_data():
#     """
#     Loads and cleans movie data from MongoDB (movie_database.movie_info).
#     """
#     df = fetch_collection_data(movie_db, "movie_info")

#     # Convert genres from list of dictionaries to comma-separated string
#     if 'genres' in df.columns:
#         def extract_genre_names(genre_data):
#             """Helper function to extract genre names correctly."""
#             if isinstance(genre_data, str):  # Handle stringified JSON
#                 try:
#                     genre_data = ast.literal_eval(genre_data)
#                 except (ValueError, SyntaxError):
#                     return ""
#             if isinstance(genre_data, list):
#                 return ', '.join([genre.get('name', '') for genre in genre_data])
#             return ""

#         df['genres'] = df['genres'].apply(extract_genre_names)

#     # Select relevant columns
#     return df[['adult', 'genres', 'release_date', 'original_language']]
import ast

def clean_movie_data():
    """
    Loads and cleans movie data from MongoDB (movie_database.movie_info).
    """
    df = fetch_collection_data(movie_db, "movie_info")

    # Ensure movie_id exists; MongoDB might not store it explicitly
    if 'movie_id' not in df.columns:
        df.reset_index(inplace=True)  # Reset index in case movie_id is missing
        df.rename(columns={'index': 'movie_id'}, inplace=True)  # Assign unique ID if missing

    # Convert genres from list of dictionaries to comma-separated string
    if 'genres' in df.columns:
        def extract_genre_names(genre_data):
            """Helper function to extract genre names correctly."""
            if isinstance(genre_data, str):  # Handle stringified JSON
                try:
                    genre_data = ast.literal_eval(genre_data)
                except (ValueError, SyntaxError):
                    return ""
            if isinstance(genre_data, list):
                return ', '.join([genre.get('name', '') for genre in genre_data])
            return ""

        df['genres'] = df['genres'].apply(extract_genre_names)

    # Ensure movie_id is in string format for consistency
    df['movie_id'] = df['movie_id'].astype(str)

    return df[['movie_id', 'adult', 'genres', 'release_date', 'original_language']]





In [32]:
def clean_watch_data():
    """
    Loads and cleans watch history data from MongoDB (movie_database.user_watch_data).
    """
    df = fetch_collection_data(movie_db, "user_watch_data")

    # Fix movie_id formatting (replace '+' with space)
    df['movie_id'] = df['movie_id'].astype(str)

    # Extract numeric watch time from "minute_mpg"
    if 'minute_mpg' in df.columns:
        df['watch_time'] = df['minute_mpg'].str.extract(r'(\d+)').astype(float)
    else:
        raise KeyError("Column 'minute_mpg' is missing from the dataset!")

    # Normalize watch time to a rating scale (1-5) instead of (0-5)
    min_watch = df['watch_time'].min()
    max_watch = df['watch_time'].max()
    df['rating'] = 1.0 + (df['watch_time'] - min_watch) / (max_watch - min_watch) * 4  # Scale between 1 and 5

    return df[['user_id', 'movie_id', 'rating']]





In [46]:
# Load and clean data from MongoDB
movies_df = clean_movie_data()
users_df = clean_user_data()
ratings_df = clean_ratings_data()
watch_df = clean_watch_data()

# Ensure column names are aligned before merging
ratings_df.rename(columns={'score': 'rating'}, inplace=True)

# Concatenate explicit ratings with normalized watch-time ratings
combined_ratings_df = pd.concat([ratings_df, watch_df], ignore_index=True)

# Fix inplace warning: Assign back instead of modifying in-place
combined_ratings_df['rating'] = combined_ratings_df['rating'].fillna(2.5)  # Default to neutral rating if missing


# Drop duplicate (user_id, movie_id) rows, keeping explicit ratings first
combined_ratings_df.sort_values(by=['user_id', 'movie_id', 'rating'], ascending=[True, True, False], inplace=True)
combined_ratings_df.drop_duplicates(subset=['user_id', 'movie_id'], keep='first', inplace=True)




In [47]:
movies_df[100:110]

,movie_id,adult,genres,release_date,original_language
100,young+guns+1988,False,"Crime, Action, Adventure, Drama, Western",1988-08-12,en
101,the+truman+show+1998,False,"Comedy, Drama",1998-06-04,en
102,the+40+year+old+virgin+2005,False,"Comedy, Romance",2005-08-11,en
103,sink+the+bismarck+1960,False,"Action, Drama, History, War",1960-02-11,en
104,if+looks+could+kill+1991,False,"Adventure, Action, Comedy",1991-03-15,en
105,the+matrix+1999,False,"Action, Science Fiction",1999-03-30,en
106,sabrina+1995,False,"Comedy, Romance",1995-12-15,en
107,barbie+a+fashion+fairytale+2010,False,"Family, Animation",2010-09-14,en
108,the+wicker+man+1973,False,"Horror, Music, Mystery, Thriller",1973-12-01,en
109,the+identical+2014,False,"Music, Drama",2014-09-05,en


In [48]:
users_df.head()

,age,occupation,gender
0,27,sales/marketing,M
1,32,college/grad student,M
2,30,sales/marketing,F
3,25,college/grad student,M
4,30,college/grad student,M


In [49]:
ratings_df.head()

,user_id,movie_id,rating
0,71467,interstellar+2014,4.0
1,17425,milk+and+honey+2003,4.0
2,29738,hamlet+1996,3.0
3,136382,a+brief+history+of+time+1991,4.0
4,123892,edmond+2005,2.0


In [50]:
watch_df[100:110]


,user_id,movie_id,rating
100,117178,my+family+1995,1.415479
101,55249,the+terminator+1984,1.260692
102,55193,naked+childhood+1968,1.301426
103,64132,eye+of+god+1997,1.407332
104,58528,young+guns+1988,1.610998
105,28129,the+truman+show+1998,1.293279
106,141752,the+40+year+old+virgin+2005,1.635438
107,30685,sink+the+bismarck+1960,1.382892
108,90194,if+looks+could+kill+1991,1.334012
109,166416,the+matrix+1999,1.162933


In [42]:
combined_ratings_df.head()

,user_id,movie_id,rating
738,100029,constantine+2005,1.244399
800,100075,cinderella+ii+dreams+come+true+2002,1.154786
2647,100108,fatherland+1994,1.521385
3206,100179,santa+sangre+1989,1.684318
1282,100236,the+lord+of+the+rings+the+return+of+the+king+2003,1.839104


In [36]:
# Ensure user_id and movie_id are strings for Surprise compatibility
combined_ratings_df['user_id'] = combined_ratings_df['user_id'].astype(str)
combined_ratings_df['movie_id'] = combined_ratings_df['movie_id'].astype(str)

# Step 1: Load Data into Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(combined_ratings_df[['user_id', 'movie_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

# Step 2: Train SVD Model
svd_model = SVD()
svd_model.fit(trainset)

# Step 3: Evaluate Model
eval_results = cross_validate(svd_model, data, cv=5, verbose=True)

print("\nEvaluation Metrics:")
for key, values in eval_results.items():
    print(f"{key}: {np.mean(values):.4f}")

# Step 4: Save the Trained Model
with open("svd_model.pkl", "wb") as f:
    pickle.dump(svd_model, f)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3556  0.3415  0.4213  0.3355  0.4390  0.3786  0.0430  
MAE (testset)     0.2614  0.2630  0.2904  0.2609  0.2948  0.2741  0.0152  
Fit time          0.03    0.03    0.03    0.03    0.03    0.03    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    

Evaluation Metrics:
test_rmse: 0.3786
test_mae: 0.2741
fit_time: 0.0276
test_time: 0.0023


In [38]:
def get_recommendations(user_id=None, num_recommendations=20):
    """
    Generate movie recommendations for a given user.
    If no user_id is provided, a random user from the dataset is selected.
    """
    if user_id is None or str(user_id) not in combined_ratings_df['user_id'].unique():
        user_id = str(np.random.choice(combined_ratings_df['user_id'].unique()))
        print(f"User ID not found, using random user: {user_id}")

    # Get movies the user has already rated
    rated_movies = set(combined_ratings_df[combined_ratings_df['user_id'] == user_id]['movie_id'].unique())

    # Get all movie IDs available
    all_movies = set(combined_ratings_df['movie_id'].unique())

    # Get movies not rated by user
    movies_to_predict = list(all_movies - rated_movies)

    if len(movies_to_predict) == 0:
        print(f"No new movies to recommend for User {user_id}. Showing random popular movies.")
        return movies_df.sample(num_recommendations)[['movie_id', 'genres']]

    # Predict ratings for unseen movies
    predictions = [svd_model.predict(user_id, movie_id) for movie_id in movies_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get top recommended movies
    top_movies = [pred.iid for pred in predictions[:num_recommendations]]
    recommended_movies = movies_df[movies_df['movie_id'].isin(top_movies)][['movie_id', 'genres']]

    if recommended_movies.empty:
        print(f"No recommendations found for User {user_id}. Check data alignment.")
    else:
        print(f"\nTop {num_recommendations} recommendations for User {user_id}:")
        print(recommended_movies)

    return recommended_movies

# Example usage
get_recommendations(user_id=66358, num_recommendations=20)



Top 10 recommendations for User 66358:
                                               movie_id  \
4     the+lord+of+the+rings+the+return+of+the+king+2003   
61                                    interstellar+2014   
84                   heimat+a+chronicle+of+germany+1984   
137                            saving+private+ryan+1998   
225                                 milk+and+honey+2003   
492                        a+brief+history+of+time+1991   
734                                           fame+2009   
806             molokai+the+story+of+father+damien+2002   
962                          berlin+alexanderplatz+1980   
1247                                         shoah+1985   

                                 genres  
4            Adventure, Fantasy, Action  
61    Adventure, Drama, Science Fiction  
84                       Drama, History  
137                 Drama, History, War  
225                               Drama  
492                         Documentary  
734       Music,

,movie_id,genres
4,the+lord+of+the+rings+the+return+of+the+king+2003,"Adventure, Fantasy, Action"
61,interstellar+2014,"Adventure, Drama, Science Fiction"
84,heimat+a+chronicle+of+germany+1984,"Drama, History"
137,saving+private+ryan+1998,"Drama, History, War"
225,milk+and+honey+2003,Drama
492,a+brief+history+of+time+1991,Documentary
734,fame+2009,"Music, Comedy, Drama, Romance"
806,molokai+the+story+of+father+damien+2002,"Drama, Foreign"
962,berlin+alexanderplatz+1980,Drama
1247,shoah+1985,Documentary


In [ ]:
import pickle
import os

# Assuming 'model' is your trained SVD instance
model_filename = "svd_model.pkl"

# Save model
# Changed 'model' to 'svd_model' to save the trained SVD model.
with open(model_filename, "wb") as f:
    pickle.dump(svd_model, f)

# Check file size
file_size = os.path.getsize(model_filename) / (1024 * 1024)  # Convert to MB
print(f"Model Size: {file_size:.2f} MB")

Model Size: 7.48 MB


In [ ]:
import sys

# Changed 'model' to 'svd_model' to get the size of the trained SVD model.
model_size = sys.getsizeof(svd_model) / (1024 * 1024)  # Convert to MB
print(f"Model Memory Usage: {model_size:.2f} MB")

Model Memory Usage: 0.00 MB


In [ ]:
import pickle
import psutil

with open("svd_model.pkl", "rb") as f:
    model = pickle.load(f)  # Load the model back into memory

process = psutil.Process()
memory_usage = process.memory_info().rss / (1024 * 1024)  # Convert to MB
print(f"Total Memory Usage: {memory_usage:.2f} MB")


Total Memory Usage: 262.93 MB


In [ ]:
import pickle

with open("svd_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

print(type(loaded_model))

<class 'surprise.prediction_algorithms.matrix_factorization.SVD'>
